<a href="https://colab.research.google.com/github/Chris-Congleton/MSc-Thesis/blob/main/Analysis/2021_SourceClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports


In [1]:
!pip install -U gensim
!pip install shap
!pip install datasets

import json, os
import pandas as pd
import numpy as np
import nltk
import glob
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm
nltk.download('punkt')

# regular expression library
import re
import pprint

import gensim
from gensim.utils import simple_preprocess
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('dutch')

#Visualisation
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction                      
import IPython

from gensim.parsing.preprocessing import preprocess_string
from gensim import utils
import gensim.models

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier

import datasets

     |████████████████████████████████| 24.1 MB 9.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 564 kB 5.5 MB/s 
     |████████████████████████████████| 325 kB 5.2 MB/s 
     |████████████████████████████████| 136 kB 34.2 MB/s 
     |████████████████████████████████| 1.1 MB 28.7 MB/s 
     |████████████████████████████████| 212 kB 32.4 MB/s 
     |████████████████████████████████| 77 kB 2.8 MB/s 
     |████████████████████████████████| 127 kB 39.2 MB/s 
     |████████████████████████████████| 144 kB 43.2 MB/s 
     |████████████████████████████████| 271 kB 48.7 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take 

# Loading and cleaning data

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
!dir gdrive/MyDrive/CS/Thesis/Testset_2021/

HetParool_2021.json  LDA_HetParool.png	LDA_Trouw.png
LDA_all.html	     LDA__.html		LDA_Volkskrant.html
LDA_All.html	     LDA_NRC.html	NRC_2021.json
LDA_all.png	     LDA_NRC.png	Telegraaf_2021.json
LDA_All.png	     LDA__.png		Trouw_2021.json
LDA_HetParool.html   LDA_Trouw.html	Volkskrant_2021.json


In [4]:
path_NRC = 'gdrive/MyDrive/CS/Thesis/Testset_2021/NRC_2021.json'
path_Trouw = 'gdrive/MyDrive/CS/Thesis/Testset_2021/Trouw_2021.json'
path_HetParool = 'gdrive/MyDrive/CS/Thesis/Testset_2021/HetParool_2021.json'
path_Telegraaf = 'gdrive/MyDrive/CS/Thesis/Testset_2021/Telegraaf_2021.json'
path_Volkskrant = 'gdrive/MyDrive/CS/Thesis/Testset_2021/Volkskrant_2021.json'

In [5]:
df_NRC = pd.read_json(path_NRC)
df_Trouw = pd.read_json(path_Trouw)
df_HetParool = pd.read_json(path_HetParool)
df_Telegraaf = pd.read_json(path_Telegraaf)
df_Volkskrant = pd.read_json(path_Volkskrant)

In [6]:
df_NRC["Source"] = "NRC"
df_Trouw["Source"] = "Trouw"
df_HetParool["Source"] = "HetParool"
df_Telegraaf["Source"] = "Telegraaf"
df_Volkskrant["Source"] = "Volkskrant"

In [7]:
print(df_NRC.shape)
print(df_Trouw.shape)
print(df_HetParool.shape)
print(df_Volkskrant.shape)

(38334, 7)
(21077, 7)
(18551, 7)
(25773, 7)


## Concat data

In [8]:

dfs = [df_NRC, df_Trouw, df_HetParool, df_Volkskrant]
df = pd.concat(dfs, ignore_index=True)

df.head()

,url,timestamp,title,publisherID,cleantext,category,Source
0,https://www.nrc.nl/nieuws/2021/01/31/een-herha...,2021-01-31 20:18:43,Een herhaling van vorig seizoen is zo goed als...,Fabian van der Poll,De eerste maand van 2021 is uitstekend verlope...,None,NRC
1,https://www.nrc.nl/nieuws/2021/01/31/avondklok...,2021-01-31 20:02:10,‘Avondklok waarschijnlijk niet verlengd als be...,Steven Musch,De avondklok wordt waarschijnlijk niet verleng...,None,NRC
2,https://www.nrc.nl/nieuws/2021/01/31/school-op...,2021-01-31 19:58:21,"School open? Dan testen, ventilatie en mondkapjes",Karel Berkhout,Door de heropening van de scholen loopt het aa...,None,NRC
3,https://www.nrc.nl/nieuws/2021/01/31/scholen-o...,2021-01-31 19:45:18,"De basisscholen gaan open, met een paar nieuwe...",Claudia Kammer,"Het werd alom verwacht, en zondag aan het ei...",None,NRC
4,https://www.nrc.nl/nieuws/2021/01/31/arrestati...,2021-01-31 19:43:49,Arrestatie Russische betoger,None,,None,NRC


## Remove missing text

In [9]:
df = df[df['cleantext'].notna()]
print(df.shape)

(103723, 7)


# Source Classifier

## Preprocessing

In [10]:
# Select only political articles

articles = df.drop(columns=['url', 'publisherID', 'category',], axis=1)
articles = articles[articles['cleantext'].notna()]
# Remove punctuation
articles['cleantext_processed'] = articles['cleantext'].map(lambda x: re.sub('[,\\.!?]', '', x))
# Save D66
articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('D66', 'dzeszes', x))
# Save BIJ1
articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('BIJ1', 'bijeeen', x))
# Save 50PLUS
articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('50PLUS', 'vijfplus', x))
# Save JA21
articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('JA21', 'jatweeeen', x))
# Convert the titles to lowercase
articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: x.lower())

p_and_p =  ['vvd', 'dzeszes', 'pvv', 'cda', 'pvda', 'sp', 'gl', 'fvd', 'pvdd',
            'cu', 'sgp', 'volt', 'jatweeeen', 'bbb', 'bijeeen', 'vijfplus', 'rutte', 'kaag',
            'wilders', 'hoekstra', 'ploumen', 'marijnissen', 'klaver', 'baudet',
            'ouwehand', 'segers', 'staaij', 'dassen', 'eerdmans', 'plas',
            'simons', 'den haan']

pat = '|'.join(r"\b{}\b".format(x) for x in p_and_p)
articles = articles[articles['cleantext_processed'].str.contains(pat)]


In [11]:
# Remove stopwords

stop_words.extend(['nbsp', 'rsquo', 'lsquo','euml', 'rdquo', 'quot', 'ldquo',
                    'we', 'wel', 'jaar', 'zegt', 'gaat','moeten', 'volgens',
                    'heel', 'maken', 'weer', 'gaan', 'twee', 'mensen', 'waar', 'alleen',
                    'komt', 'zoals', 'alle', 'wij', 'zei','komen','tussen', 'eerste','zien'
                    'eigen', 'leven','grote', 'nieuwe','mee','goed', 'aantal','laten'
                    ,'tweede', 'af', 'eerder', 'eigen', 'elkaar','afgelopen', 'week',
                    'zeggen', 'echt', 'dag', 'zien', 'steeds', 'laten', 'willen', 'waarin',
                   'duizend','voormalig', 'nu', 'we'])

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
            if word not in stop_words] for doc in texts]

articles['cleantext_processed'] = remove_stopwords(articles['cleantext_processed'])

articles['cleantext_processed'] = [' '.join(map(str, l)) for l in articles['cleantext_processed']]

In [12]:
print(articles['cleantext_processed'].head())

1     avondklok waarschijnlijk verlengd februari zol...
3     alom verwacht zondag eind bevestigden demissio...
6     wantrouwen overheid spuit eruit eindelijk ligt...
15    lang gedacht stijging zeespiegel einde eeuw me...
22    basisscholen kinderopvang vanaf maandag februa...
Name: cleantext_processed, dtype: object


In [13]:
Xy = articles.drop(columns=['timestamp', 'title'])
X = Xy['cleantext_processed']
y = Xy['Source']

In [14]:
print(X.shape)

(15498,)


In [15]:
print(y.shape)

(15498,)


# RobBert

In [16]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 43.3 MB/s 
     |████████████████████████████████| 596 kB 50.2 MB/s 
     |████████████████████████████████| 6.5 MB 34.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [17]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base")

Downloading:   0%|          | 0.00/715k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/374k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/660 [00:00<?, ?B/s]

In [18]:
articles = df.drop(columns=['url', 'publisherID', 'category','timestamp', 'title'], axis=1)
articles = articles[articles['cleantext'].str.len()>5]
df_bert = articles

In [19]:
df_bert.head()

,cleantext,Source
0,De eerste maand van 2021 is uitstekend verlope...,NRC
1,De avondklok wordt waarschijnlijk niet verleng...,NRC
2,Door de heropening van de scholen loopt het aa...,NRC
3,"Het werd alom verwacht, en zondag aan het ei...",NRC
5,"Het was lang wachten, maar 36 jaar nadat Jan T...",NRC


In [20]:
def preprocess_function(examples):
    return tokenizer(examples['cleantext'], truncation=True, padding=True, max_length=128)

In [21]:
Xy_dd = datasets.Dataset.from_pandas(df_bert)

In [22]:
Xy_dd = Xy_dd.train_test_split(test_size=0.2)

In [23]:
tokenized = Xy_dd.map(preprocess_function, batched=True)

  0%|          | 0/78 [00:00<?, ?ba/s]

  0%|          | 0/20 [00:00<?, ?ba/s]

In [24]:
print(tokenized)

DatasetDict({
    train: Dataset({
        features: ['cleantext', 'Source', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 77472
    })
    test: Dataset({
        features: ['cleantext', 'Source', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 19368
    })
})


In [25]:
tokenized = tokenized.class_encode_column("Source")

Casting to class labels:   0%|          | 0/78 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/8 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/20 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

In [26]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [27]:
from transformers import TrainingArguments, Trainer

In [28]:
model = RobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base", num_labels=4)

Downloading:   0%|          | 0.00/446M [00:00<?, ?B/s]

Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.out_proj.weight', 'cl

In [29]:
tokenized = tokenized.rename_column("Source", "label")
print(tokenized)

DatasetDict({
    train: Dataset({
        features: ['cleantext', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 77472
    })
    test: Dataset({
        features: ['cleantext', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 19368
    })
})


In [30]:
!pip install wandb

import wandb
wandb.login()

     |████████████████████████████████| 1.8 MB 5.4 MB/s 
     |████████████████████████████████| 181 kB 44.5 MB/s 
     |████████████████████████████████| 144 kB 29.7 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=d088bfcdf047baa88fec7c5f7b5c53f5e4560e9efd4a055dd2f27bc80a0c22c9
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [31]:
%env WANDB_PROJECT=source_classification

env: WANDB_PROJECT=source_classification


In [32]:
from datasets import load_metric

In [35]:
def compute_metrics(eval_pred):
    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("f1")
    metric4 = load_metric("accuracy")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = metric1.compute(predictions=predictions, references=labels, average="micro")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="micro")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="micro")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]

    return {"precision": precision, "recall": recall, "f1":f1, "accuracy":accuracy}

In [36]:
training_args = TrainingArguments(

    output_dir="./results",

    learning_rate=1e-5,

    per_device_train_batch_size=16,

    per_device_eval_batch_size=16,

    num_train_epochs=1,

    weight_decay=0.01,

    report_to="wandb",

    evaluation_strategy = 'steps',

    logging_steps = 100,                    # we will log every 100 steps
    eval_steps = 1000,                      # we will perform evaluation every 500 steps
    save_steps = 1000,
    load_best_model_at_end = True,

)

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=tokenized['train'],

    eval_dataset=tokenized['test'],

    tokenizer=tokenizer,

    data_collator=data_collator,

    compute_metrics=compute_metrics,

)

trainer.train()

PyTorch: setting up devices
The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, cleantext. If __index_level_0__, cleantext are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 77472
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4842
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
10,1.049500,0.917145,0.586225,0.586225,0.586225,0.586225
20,0.909700,0.784520,0.608581,0.608581,0.608581,0.608581
30,0.772500,0.750374,0.618907,0.618907,0.618907,0.618907
40,0.735700,0.687083,0.712102,0.712102,0.712102,0.712102
50,0.767100,0.660303,0.678542,0.678542,0.678542,0.678542
60,0.660200,0.598099,0.690056,0.690056,0.690056,0.690056
70,0.601400,0.586792,0.716749,0.716749,0.716749,0.716749
80,0.596800,0.600618,0.728521,0.728521,0.728521,0.728521
90,0.557800,0.563862,0.729657,0.729657,0.729657,0.729657
100,0.572900,0.543324,0.727334,0.727334,0.727334,0.727334


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, cleantext. If __index_level_0__, cleantext are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 19368
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-10
Configuration saved in ./results/checkpoint-10/config.json
Model weights saved in ./results/checkpoint-10/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-10/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-10/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, cleantext. If __index_level_0__, cleantext are not expected by `RobertaForSequenceClassification.forward`,  you can s

RuntimeError: ignored

In [ ]:
wandb.finish()

# BERTje

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base")

tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")

In [ ]:
Xy_dd = datasets.Dataset.from_pandas(df_bert)

In [ ]:
Xy_dd = Xy_dd.train_test_split(test_size=0.2)

In [ ]:
tokenized = Xy_dd.map(preprocess_function, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base", num_labels=4)

In [ ]:
tokenized = tokenized.rename_column("Source", "label")
print(tokenized)

In [ ]:
training_args = TrainingArguments(

    output_dir="./results",

    learning_rate=2e-5,

    per_device_train_batch_size=16,

    per_device_eval_batch_size=16,

    num_train_epochs=1,

    weight_decay=0.01,

)

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=tokenized['train'],

    eval_dataset=tokenized['test'],

    tokenizer=tokenizer,

    data_collator=data_collator,

    compute_metrics=compute_metrics,

    report_to="wandb",  # enable logging to W&B

)

trainer.train()

In [ ]:
wandb.finish()

# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7)
X = tfidfconverter.fit_transform(X).toarray()

# Decision Tree Classifier

In [ ]:


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_encoded_input, y, test_size=0.2, random_state=0)

NameError: ignored

In [ ]:

classifier = DecisionTreeClassifier(max_depth=20,random_state=17)
classifier.fit(X_train, y_train) 

y_pred = classifier.predict(X_test)



In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

In [ ]:
features = list(tfidfconverter.get_feature_names_out())


In [ ]:
from sklearn import tree
text_representation = tree.export_text(classifier, feature_names=features)
print(text_representation)

# SVM

In [ ]:
from sklearn import svm

In [ ]:
clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

# xgboost


In [ ]:
import xgboost
model = xgboost.XGBClassifier().fit(X_train, y_train)
y_pred = model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))